In [1]:
#!pip install transformers[sentencepiece]
#!pip install -U datasets
#!pip install sacremoses
#!pip install sacrebleu
import transformers

from datasets import load_dataset

raw_datasets = load_dataset("kde4", lang1="en", lang2="ru")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.10k [00:00<?, ?B/s]

kde4.py:   0%|          | 0.00/4.25k [00:00<?, ?B/s]

In [2]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets["train"][1]["translation"]

{'en': 'Destination:', 'ru': 'Место назначения:'}

In [3]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-ru"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

In [4]:
en_sentence = split_datasets["train"][1]["translation"]["en"]
ru_sentence = split_datasets["train"][1]["translation"]["ru"]

inputs = tokenizer(en_sentence, text_target=ru_sentence)

wrong_targets = tokenizer(ru_sentence)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

['▁', 'М', 'е', 'с', 'т', 'о', '▁', 'н', 'а', 'з', 'н', 'а', 'ч', 'е', 'н', 'и', 'я', ':', '</s>']
['▁Место', '▁назначения', ':', '</s>']


In [ ]:
max_length = 128


def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["ru"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [ ]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

In [ ]:
#!pip install tf-keras
from transformers import AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
import tf_keras
#!pip uninstall jupyterlab_widgets
#!pip install jupyterlab_widgets





model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from transformers import Seq2SeqTrainingArguments
#!pip install transformers[torch]

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-ru",
    eval_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], collate_fn=data_collator, batch_size=8
)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
#!pip install transformers
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
#from huggingface_hub import Repository, get_full_repo_name

#model_name = "diploma-test-kde4-en-to-ru-accelerate"
#repo_name = get_full_repo_name(model_name)


In [ ]:
#output_dir = "diploma-test-kde4-en-to-ru-accelerate"
#repo = Repository(output_dir, clone_from=repo_name)

In [ ]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # заменяем -100, не можем прочесть
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # тренируем
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # оценка
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    # сохраняем и загружаем на обнимающлицо
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

In [65]:
#from huggingface_hub import notebook_login

#notebook_login()
#!pip install Cmake
#!pip install torch
from transformers import AutoTokenizer, MarianMTModel

src = "en"  # source language
trg = "ru"  # target language

model_name = f"Helsinki-NLP/opus-mt-{src}-{trg}"
model = MarianMTModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)



In [77]:
#!pip install evaluate
import evaluate
from nltk.tokenize import word_tokenize
import nltk
import sacrebleu
#nltk.download('punkt_tab')
bleu = evaluate.load("bleu")

sample_text = "i have to stand up, i cant stand sitting on this stand" #текст, подаваемый на перевод
reference_text= "мне нужно встать, я терпеть не могу сидеть на этой стойке" #эталонный перевод

batch = tokenizer([sample_text], return_tensors="pt")
generated_ids = model.generate(**batch)
model_translated = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(model_translated)  #перевод от модели

prediction = [word_tokenize(model_translated.lower())] #токенизация для рассчёта BLEU
#reference = [[word_tokenize(reference_text.lower())]]

#results = bleu.compute(predictions=model_translated.lower(), references=reference_text)  # Оценка переведённого текста в BLEU
score = sacrebleu.corpus_bleu(model_translated, reference_text)
print(score)

#sample_text = "i have to stand up, i cant stand sitting on this stand" #текст, подаваемый на перевод
#reference_text= "мне нужно встать, я терпеть не могу сидеть на этой стойке"  #эталонный перевод

Я должен встать, я не могу стоять на этой стойке.
BLEU = 0.00 0.0/0.0/0.0/0.0 (BP = 1.000 ratio = 1.000 hyp_len = 1 ref_len = 1)
